In [ ]:

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split

import os
print(os.listdir("../input"))



In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.columns

In [ ]:
train.shape

In [ ]:
#Data exploration
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train['SalePrice'].describe()

In [ ]:
sns.distplot(train['SalePrice']);

In [ ]:
print("Skewness: %f" % train['SalePrice'].skew())
print("Kurtosis: %f" % train['SalePrice'].kurt())

In [ ]:
g = sns.pairplot(train, vars=["SalePrice", "GrLivArea"])

In [ ]:
sns.pairplot(train, vars=["SalePrice", "TotalBsmtSF"])

In [ ]:
sns.pairplot(train, vars=["SalePrice", "OverallQual"])

In [ ]:
sns.pairplot(train, vars=["SalePrice", "OverallQual", "GrLivArea","TotalBsmtSF"])

In [ ]:
sns.boxplot( x = "OverallQual", y = "SalePrice",data = train)

In [ ]:

f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot( x = "YearBuilt", y = "SalePrice",data = train)
plt.xticks(rotation=90)

In [ ]:
train.corr()


In [ ]:
train.corr()
f, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(train.corr(), vmax=.8, square=True);

In [ ]:
k = 10 #number of variables for heatmap
cols = train.corr().nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], size = 2.5)
plt.show();

In [ ]:
train.head()

In [ ]:
#convert categorical variable into dummy
train = pd.get_dummies(train)

In [ ]:
test = pd.get_dummies(test)

In [ ]:
train.head()

In [ ]:
test.head(7)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# pull data into target (y) and predictors (X)
train_y = train.SalePrice
predictor_cols = ['LotArea', 'OverallQual','OverallCond','YearBuilt', 'TotRmsAbvGrd', 'GrLivArea','GarageArea','TotalBsmtSF','1stFlrSF'
                 , 'FullBath', 'HalfBath','LotFrontage','MSSubClass', 'YearRemodAdd','BsmtFinSF1','BsmtUnfSF','2ndFlrSF', 'BedroomAbvGr','Fireplaces', 'WoodDeckSF',
                  'OpenPorchSF','MoSold']

# Create training predictors data
#train_X = train[predictor_cols]

#model = RandomForestRegressor(n_estimators=50, random_state=1)
#model.fit(train_X, train_y)

In [ ]:
train[predictor_cols].head()

In [ ]:
np.any(np.isnan(train[predictor_cols]))

In [ ]:
np.any(np.isnan(test[predictor_cols]))

In [ ]:
train[predictor_cols] = train[predictor_cols].fillna(train[predictor_cols].mean())

In [ ]:
test[predictor_cols] = test[predictor_cols].fillna(test[predictor_cols].mean())

In [ ]:
np.any(np.isnan(train[predictor_cols]))

In [ ]:
np.any(np.isnan(test[predictor_cols]))

In [ ]:
# Create training predictors data
train_X = train[predictor_cols]
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import plot_importance
#model = RandomForestRegressor(n_estimators=50, random_state=1)
#model.fit(train_X, train_y)
#from sklearn.model_selection import GridSearchCV

#xgb_model = XGBRegressor()
#optimization_dict = {'max_depth': [2,4,6],
  #                   'n_estimators': [50,100,200, 500, 1000, 2000]}

#model = GridSearchCV(xgb_model, optimization_dict, 
    #               verbose=1)

#model.fit(train_X, train_y)
#print(model.best_score_)
#print(model.best_params_)



xgb = XGBRegressor(n=1500)
xgb.fit(train_X, train_y, verbose=False)

In [ ]:
# Read the test data
#test = pd.read_csv('../input/test.csv')
# Treat the test data in the same way as training data. In this case, pull same columns.
test_X = test[predictor_cols]
#model=Lasso(alpha =0.001, random_state=1)

#model.fit(train_X,train_y)

#pred=model.predict(test_X)
#print(pred)
# Use the model to make predictions
predicted_prices = xgb.predict(test_X)
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_prices)

In [ ]:


plot_importance(xgb, max_num_features=10)



In [ ]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission6.csv', index=False)
